<a href="https://colab.research.google.com/github/jkf87/CNU-industrialAI/blob/main/%5B%EC%98%88%EC%A0%9C%5D%EB%82%98%EC%B2%98%EB%9F%BC_%EB%A7%90%ED%95%98%EB%8A%94_%EC%B1%97%EB%B4%87_%EB%A7%8C%EB%93%A4%EA%B8%B0_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##필요한 라이브러리 설치하기
- [openai](https://platform.openai.com/docs/introduction)
- [gradio](https://www.gradio.app/docs/interface)
- 최근 업데이트 이후로 레거시 코드를 지원하지 않는 경우가 있어 코드에 오류가 발생했을경우  [마이그레이션 가이드](https://learn.microsoft.com/ko-kr/azure/ai-services/openai/how-to/migration?tabs=python%2Cdalle-fix) 를 참고하세요

In [ ]:
!pip install -q openai gradio

In [ ]:
# @title 안되면 레거시 코드를 씁니다.

!pip install -q openai==0.28

#나처럼 말하는 챗봇 만들기
- 카카오톡 데이터를 내보내기 하여 txt파일을 받아주세요.
![image.png](https://i.imgur.com/XGVLdJV.png)
- 2명의 대화만 지원합니다.
- 3명 이상의 대화를 처리하려면 `user`와 `asistant`를 지정해서 1:1의 대화 형태로 별도로 대화를 출력해 finetune할 수 있습니다
- `system_message_content`에 입력된 프롬프트를 수정해주세요


In [ ]:
from google.colab import files
import re
import json

def upload_kakao_file():
    print("카카오톡 데이터 파일을 업로드 해주세요.")
    uploaded = files.upload()
    for file_name in uploaded.keys():
        return file_name  # Return the name of the uploaded file

def modified_kakao_to_jsonl_and_save() -> None:
    file_path = upload_kakao_file()

    # Prompt the user for names
    user_name = input("user_name에 사용할 이름을 입력해주세요 (예: 홍길동): ")
    assistant_name = input("assistant_name에 사용할 이름을 입력해주세요 (예: 한준구): ")

    with open(file_path, "r", encoding="utf-8") as file:
        kakao_data = file.read()

    date_splits = re.split(r'--------------- \d{4}년 \d{1,2}월 \d{1,2}일 [월화수목금토일]요일 ---------------', kakao_data)

    jsonl_data = []
    for date_split in date_splits:
        day_messages = re.findall(rf'\[({assistant_name}|{user_name})\] \[.+\] (.+)', date_split)

        # If there are any messages for this date
        if day_messages:
            for i in range(len(day_messages) - 1):
                # 현재 메시지와 다음 메시지 쌍을 확인
                current_message = day_messages[i]
                next_message = day_messages[i + 1]

                # 사용자의 메시지 다음에 어시스턴트의 메시지가 오는 경우만 처리
                if current_message[0] == user_name and next_message[0] == assistant_name:
                    conversation_entry = {
                        "messages": [
                            {"role": "system", "content":  f"너는 카카오톡 대화를 하는 60대 여성이고 이름은 {assistant_name}야, 사람처럼 행동해"},
                            {"role": "user", "content": current_message[1]},
                            {"role": "assistant", "content": next_message[1]}
                        ]
                    }
                    jsonl_data.append(json.dumps(conversation_entry, ensure_ascii=False))

    # Save the converted data to a .jsonl file
    output_file_path = "converted_kakao_data.jsonl"
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write("\n".join(jsonl_data))

    print(f"변환된 데이터가 '{output_file_path}'로 저장되었습니다.")

# Execution in Colab
modified_kakao_to_jsonl_and_save()


#파인튜닝을 위해 데이터 업로드하기
- openai api 키 입력
  - `openai.api_key` 에 openai api key를 입력합니다. 모둠별 api key를 돌아가면서 한명씩 실습하도록 하겠습니다.
- 데이터 업로드
  - 앞에서 생성한 `json` 데이터 파일을 업로드 해주세요.

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key='sk-',
)


client.files.create(
    file=open("/content/converted_kakao_data.jsonl", "rb"),
    purpose='fine-tune'
)

### 파일 업로드 확인하기

- 내가 올린 파일 이름 `"filename"`을 찾기
- `file-영어대소문자` 형태 이름을 확인
- `"status": "processed",` 인지 확인하기 (업로드 완료)

In [ ]:
import json

files_list=client.files.list()

# 각 FileObject의 정보를 추출하여 리스트에 저장
files_info = []
for file_object in files_list.data:
    file_info = {
        'id': file_object.id,
        'bytes': file_object.bytes,
        'created_at': file_object.created_at,
        'filename': file_object.filename,
        'object': file_object.object,
        'purpose': file_object.purpose,
        'status': file_object.status,
        'status_details': file_object.status_details
    }
    files_info.append(file_info)

# JSON 데이터로 변환 및 예쁘게 출력
print(json.dumps(files_info, indent=4))

In [ ]:
# @title 잘못 올린 파일 삭제하기
client.files.delete("file-Xuua6O5HJIS8WwJWaIqSVy0c")

##업로드 된 데이터 id를 아래 코드에 넣기
- `"id": file-난수 ` 부분을 복사하기
- 업로드 확인
  - [업로드 여부 확인](https://platform.openai.com/files) 클릭하여 파일 업로드 되었는지 확인하기
  - 자신의 계정이 아닌 경우 `openai.files.list()` (앞 코드 활용)
- id 지정하고 model 튜닝 시작하기

In [ ]:
#작업에 맞게 id 변경하기
id = "file-ZN0Rur5Rls8KSff6rjlgi7r0"

client.fine_tuning.jobs.create(
    training_file=id,
    model="gpt-3.5-turbo"
)

## 튜닝 결과 : `ft.gpt-3.5-turbo-0613:personal:난수` 형태 모델이름 확인하기
`"fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::난수`
 - [튜닝과정확인](https://platform.openai.com/finetune)

In [ ]:
# @title 레거시 코드, 기존 진행상황을 확인하는 방법
#이걸로 진행상황을 알 수 있음
client.fine_tuning.jobs.list(limit=10)



## 파인튜닝 작업 추적하기

In [ ]:
import time
ft_id = "ftjob-39EDncDyAO4xyZa8RBCSkBjM"

## 일정 기간동안 돌아가면서 step, training loss 등을 확인

In [ ]:
import openai
import time
from datetime import datetime

# 마지막으로 처리한 이벤트의 시간을 추적
last_processed_time = 0

while True:
    # Fine-tuning 작업 상태 조회
    job_status = client.fine_tuning.jobs.retrieve(ft_id)
    if job_status.status in ['succeeded', 'failed', 'cancelled']:
        print(f"Fine-tuning 작업 완료: {job_status.status}")
        break

    # Fine-tuning 작업 이벤트 조회
    events = client.fine_tuning.jobs.list_events(ft_id).data

    # 현재 시간 기록
    current_time = datetime.now().strftime("%H시 %M분 %S초")

    for event in events:
        if event.type == 'metrics':
            event_time = event.created_at
            # 새로운 이벤트만 처리
            if event_time > last_processed_time:
                last_processed_time = event_time
                step = event.data.get('step', '-')
                train_loss = event.data.get('train_loss', '-')
                val_loss = event.data.get('val_loss', '-')
                print(f"{current_time}\tStep: {step}\tTraining Loss: {train_loss}\tValidation Loss: {val_loss}")

    # 10초 대기 후 반복
    time.sleep(10)

# Fine-tuning 작업 완료 후 추가 정보 출력
if job_status.status == 'succeeded':
    print(f"Fine-tuning 작업 완료: {job_status.status}")
    if hasattr(job_status, 'fine_tuned_model'):
        print(f"생성된 모델: {job_status.fine_tuned_model}")
else:
    print(f"Fine-tuning 작업 실패 또는 취소: {job_status.status}")

## 튜닝 모델 테스트 해보기
- 잘 튜닝되었는지 테스트
- gradio로 챗봇 서빙하기



In [ ]:

#간단하게 출력 확인하기
model_id = job_status.fine_tuned_model

user_input = "뭐라고 기도해줄꺼야?"

completion = client.chat.completions.create(
    model=model_id,
    messages=[
        {"role": "system", "content": "너는 카카오톡 대화를 하는 60대 여성이고 이름은 엄마야, 논리적인 추론 능력을 유지하며 스타일에 맞춰 이야기해."},
        {"role": "user", "content": user_input}
    ]
)

print(completion.choices[0].message.content)

In [ ]:
#gradio로 챗봇 서빙하기
import gradio as gr

def predict(input, history):
    history.append({"role": "user", "content": input})

    gpt_response = client.chat.completions.create(
        model=model_id,
        messages=history,
        temperature=0.2,  # 낮은 temperature 값으로 더 보수적인 답변을 얻습니다.
        max_tokens=200,    # 출력의 길이를 제한합니다.
        top_p=0.8,        # top_p 값을 조절하여 샘플링 전략을 변경합니다.
        frequency_penalty=1,  # 필요하다면 패널티를 적용합니다.
        presence_penalty=1,    # 필요하다면 패널티를 적용합니다.
    )

    response = gpt_response.choices[0].message.content

    history.append({"role": "assistant", "content": response})

    messages = [(history[i]["content"], history[i+1]["content"]) for i in range(1, len(history), 2)]

    return messages, history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="ChatBot")

    state = gr.State([{
        "role": "system",
        "content": "너는 카카오톡 대화를 하는 60대 여성이고 이름은 엄마야, 논리적인 추론 능력을 유지하며 스타일에 맞춰 이야기해."
    }])

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="엄마랑 대화하기")

    txt.submit(predict, [txt, state], [chatbot, state])

demo.launch(debug=True, share=True)
